In [1]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import json
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
from gezi import tqdm
import gezi

# vocab of watch vids

In [133]:
counter_train = gezi.WordCounter()
counter_eval = gezi.WordCounter()
counter = gezi.WordCounter()

In [134]:
for k in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{k + 1}/user.parquet')
  d.watch = d.watch.apply(json.loads)
  watches = d.watch.values
  for i in range(len(watches)):
    for j in range(len(watches[i])):
      counter.add(watches[i][j][1])
      counter_train.add(watches[i][j][1])

100%|██████████| 30/30 [07:25<00:00, 15.83s/it]


In [135]:
d = gezi.read_parquet('../input/eval/user.parquet')
d.watch = d.watch.apply(json.loads)
watches = d.watch.values
for i in tqdm(range(len(watches))):
  for j in range(len(watches[i])):
    counter.add(watches[i][j][1])
    counter_eval.add(watches[i][j][1])

100%|██████████| 156273/156273 [00:18<00:00, 8314.32it/s]


In [136]:
counter.total

98602040

In [137]:
len(counter.counter)

416104

In [138]:
len(counter_train.counter)

406739

In [139]:
len(counter_eval.counter)

159294

In [9]:
counter_train.counter.most_common(10)

[(2515906008, 412909),
 (4073766715, 328276),
 (1123409917, 294605),
 (2374679613, 285496),
 (3373925819, 279307),
 (1402684763, 274267),
 (1035855704, 271982),
 (307031161, 271533),
 (1117348385, 269728),
 (997539952, 268953)]

In [10]:
counter_eval.counter.most_common(10)

[(2633810051, 45240),
 (3380550257, 42657),
 (1208039942, 42385),
 (82883498, 34296),
 (2350055813, 32793),
 (2085397717, 32038),
 (3061456451, 31718),
 (894801038, 31154),
 (3844996456, 30999),
 (4150073433, 30432)]

In [11]:
counter_eval.counter[2633810051]

45240

In [12]:
counter_train.counter[2633810051]

41530

In [13]:
counter_train.counter[3380550257]

186128

In [15]:
len(counter_eval.counter - counter_train.counter)

10380

In [16]:
set(counter_eval.counter.keys()) - set(counter_train.counter.keys())

{3319955456,
 4064444418,
 4104650757,
 1704067077,
 1243545608,
 4070375433,
 4276617232,
 4021747730,
 1034321940,
 532447253,
 2226257941,
 2249064477,
 898203681,
 2495971378,
 2541912114,
 2685796413,
 429162569,
 2441510985,
 4015325261,
 2063925332,
 468320342,
 1146159207,
 2095710312,
 380403817,
 1031798890,
 3807477874,
 3197796466,
 3456598130,
 3076980855,
 3864232055,
 1824686204,
 334200962,
 3171844231,
 4059562124,
 3631841420,
 2457742,
 1843986578,
 2701983891,
 3442933910,
 2468282522,
 3328442523,
 933986460,
 3829956768,
 3338666151,
 764117159,
 4108583082,
 3385983149,
 1203437744,
 3526852786,
 1085604019,
 3304849594,
 3390767291,
 3639738559,
 4134961345,
 2674983111,
 2373484755,
 3522724053,
 3105947864,
 1196458205,
 3158278367,
 1213825252,
 1557364968,
 3343778025,
 3994878190,
 4183884015,
 273088751,
 2516877562,
 2717516028,
 1081049346,
 2316108050,
 2042626324,
 411599127,
 2408022299,
 363266334,
 1981022495,
 3932160288,
 2091909412,
 3809280292,


In [17]:
len(set(counter_eval.counter.keys()) - set(counter_train.counter.keys()))

9365

In [18]:
counter.save('../input/all/watch_vids.txt')

total_count 98602040 unknown_count 0 total_word 416104


In [19]:
counter_train.save('../input/train/watch_vids.txt')

total_count 92592550 unknown_count 0 total_word 406739


In [20]:
counter_eval.save('../input/eval/watch_vids.txt')

total_count 6009490 unknown_count 0 total_word 159294


# vocab of vids

In [130]:
counter2_train = gezi.WordCounter()
counter2_eval = gezi.WordCounter()
counter2 = gezi.WordCounter()
counter_prev = gezi.WordCounter()

In [131]:
for i in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{i+1}/context.parquet')
  for vid in d.vid.values:
    counter2.add(vid)
    counter2_train.add(vid)
  d = d[d.prev!=0]
  counter_prev.adds(d.prev)

100%|██████████| 30/30 [05:54<00:00, 12.10s/it]


In [132]:
d = gezi.read_parquet(f'../input/eval/context.parquet')
for vid in tqdm(d.vid.values):
  counter2.add(vid)
  counter2_eval.add(vid)
d = d[d.prev!=0]
counter_prev.adds(d.prev)

100%|██████████| 5551832/5551832 [00:20<00:00, 264774.42it/s]


In [24]:
len(counter2.counter)

165527

In [25]:
len(counter2_train.counter)

162719

In [26]:
len(counter2_eval.counter)

42815

In [27]:
len(counter2_eval.counter - counter2_train.counter)

3458

In [28]:
set(counter2_eval.counter.keys()) - set(counter2_train.counter.keys())

{3700940800,
 1927471105,
 114892802,
 4144521219,
 1577000964,
 1832321029,
 1343741959,
 1399963656,
 766271504,
 1072545812,
 1029578775,
 319725594,
 1170292762,
 2930483229,
 2329772073,
 4193099818,
 1436680235,
 4085588013,
 1678172206,
 714719280,
 1038401586,
 2020556853,
 2205261879,
 2526879806,
 3974389824,
 1815363651,
 2483077187,
 1556955205,
 361979974,
 71229513,
 3031597130,
 94019683,
 444031078,
 2095710312,
 3165233257,
 1945477228,
 3838181486,
 3197796466,
 2790514805,
 2833522811,
 2283577469,
 496746622,
 3772604544,
 3736436867,
 1986470020,
 172166,
 1365958790,
 1428349062,
 3580346504,
 3246309518,
 4092256402,
 3710664858,
 1978417307,
 3459637406,
 2345951393,
 1864704164,
 3898966183,
 3777183918,
 1490403503,
 4201930929,
 3389259954,
 30097594,
 1099997379,
 1416200392,
 7676105,
 3250651346,
 4218929364,
 1202233563,
 3747250407,
 563642599,
 1590485223,
 2911895788,
 2613674222,
 274718960,
 3867762936,
 1983160570,
 419602687,
 208478464,
 399026611

In [29]:
len(set(counter2_eval.counter.keys()) - set(counter2_train.counter.keys()))

2808

In [30]:
counter2.save('../input/all/context_vids.txt')

total_count 97147343 unknown_count 0 total_word 165527


In [31]:
counter2_train.save('../input/train/context_vids.txt')

total_count 91595511 unknown_count 0 total_word 162719


In [32]:
counter2_eval.save('../input/eval/context_vids.txt')

total_count 5551832 unknown_count 0 total_word 42815


In [129]:
couter_prev.save('../input/all/prev_vids.txt')

total_count 0 unknown_count 0 total_word 0


In [140]:
len(counter.counter)

416104

In [141]:
for vid, count in counter2.counter.items():
  counter.add(vid, count)
for vid, count in counter_prev.counter.items():
  counter.add(vid, count)

In [142]:
len(counter.counter)

456740

In [143]:
counter.save('../input/all/vids.txt')
counter.save('../input/all/vid.txt')

total_count 223193581 unknown_count 0 total_word 456740
total_count 223193581 unknown_count 0 total_word 456740


In [37]:
for vid, count in counter2_train.counter.items():
  counter_train.add(vid, count)

In [38]:
counter_train.save('../input/train/vids.txt')

total_count 184188061 unknown_count 0 total_word 441562


In [39]:
for vid, count in counter2_eval.counter.items():
  counter_eval.add(vid, count)

In [40]:
counter_eval.save('../input/eval/vids.txt')

total_count 11561322 unknown_count 0 total_word 168983


# stats

In [43]:
watch_vids = gezi.Vocab('../input/all/watch_vids.txt')

In [44]:
context_vids = gezi.Vocab('../input/all/context_vids.txt')

In [49]:
w_vids = set(watch_vids.reverse_vocab)

In [50]:
c_vids = set(context_vids.reverse_vocab)

In [52]:
len(w_vids & c_vids)

130634

In [53]:
len(w_vids & c_vids) / len(c_vids)

0.7891910178881042

In [54]:
len(w_vids & c_vids) / len(w_vids)

0.31394404310440127

In [55]:
# item table

In [97]:
dis = []
for k in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{k + 1}/item.parquet')
  d['day'] = k + 1
  dis += [d]
d = gezi.read_parquet('../input/eval/item.parquet')
d['day'] = 31
dis += [d]
di = pd.concat(dis)

100%|██████████| 30/30 [00:01<00:00, 24.66it/s]


In [70]:
len(set(di.vid))

178289

In [71]:
len(w_vids | c_vids)

451001

In [78]:
374341640 in c_vids

False

In [79]:
context_vids.id(374341640)

118151

In [81]:
set(di.vid.astype(str)) - (w_vids | c_vids)

{'2266392562',
 '3330812104',
 '2813697292',
 '3751447291',
 '3643331691',
 '3841242191',
 '3332128870',
 '751234477',
 '2624366093',
 '778086537',
 '1393038607',
 '113396284',
 '2453170317',
 '16735862',
 '1714382164',
 '627777174',
 '3699306564',
 '1857430398',
 '2326029917',
 '847759064',
 '3602480831',
 '3789409519',
 '2216185110',
 '1032019024',
 '3655027101',
 '188572703',
 '2770061996',
 '819698417',
 '764917742',
 '3147028572',
 '2555359178',
 '4288477297',
 '3838406812',
 '3734279687',
 '1423500783',
 '2513824708',
 '2981496402',
 '3690183844',
 '837806449',
 '836009404',
 '3596968543',
 '3137985438',
 '1818442475',
 '2466519208',
 '3049346748',
 '3568437086',
 '3002601756',
 '1321403003',
 '3031944942',
 '1188542472',
 '2579173855',
 '4062141563',
 '2111650072',
 '3077420726',
 '2188713408',
 '1684990227',
 '1536143773',
 '1334855671',
 '250315204',
 '511482848',
 '4115899054',
 '411595233',
 '1590389375',
 '2643411793',
 '3524865120',
 '488209540',
 '1545231723',
 '361261032

In [83]:
len(set(di.vid.astype(str)) - (w_vids | c_vids))

5783

In [73]:
di[di.vid==374341640]

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
42261,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,4.0,30
42843,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,4.0,30
45730,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,0.0,30
42988,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,0.0,30
44226,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,0.0,30
38866,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,4.0,30
36001,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,52.0,30
47334,374341640,58,1311081178,"[1727103792, 577373787, 225086817, 2882497719,...",3238133344,1820266297,3172394293,0,151.509995,0.0,84.0,31


In [75]:
dis[-2]

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
0,15819492,98,1407204002,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.0725,1996.0,30
1,38297769,112,1425690005,"[3115812425, 2144664037, 1221620183, 325915182...",2168442617,1413776458,472867291,0,42.509998,0.0000,5832.0,30
2,68075555,54,1370912171,[1688233050],243169446,2993616479,2234347078,0,292.119995,0.0000,4.0,30
3,72551649,28,1412650983,[],196493883,3174145329,2234347078,0,882.900024,0.0000,212.0,30
4,73710072,60,1397102895,"[2160407336, 3745672243, 325393889, 3169431763]",2056941596,1413776458,3172394293,0,25.070000,0.0000,56.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...
42944,4248901384,21,1422849565,"[3902504787, 1235418045, 3804658193]",4074303298,1820266297,2234347078,0,1387.569946,0.0000,5388.0,30
42945,4255628613,33,1413545288,[1134598214],1016220003,1820266297,2234347078,0,35.970001,0.0000,440.0,30
42946,4272496705,80,1387029762,"[3309131045, 2561243119, 184102567, 2880021111...",435329116,1820266297,472867291,0,115.540001,0.0000,104.0,30
42947,4283547666,58,1424684908,"[2142910698, 1316768238, 1853987563, 1984414365]",2753664459,470292959,2234347078,0,154.779999,0.0600,12628.0,30


In [77]:
dis[-2][dis[-2].vid==4283640696]

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
42948,4283640696,81,1367676865,"[2255039294, 3696039767, 1984414365, 193682327...",2553585254,1413776458,3172394293,0,78.480003,0.0,4036.0,30


In [98]:
di[di.vid==2266392562]

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
646,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,4.0,5
671,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,4.0,6
681,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,4.0,9
647,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,4.0,10
666,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,20.0,14
639,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,20.0,15
663,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,28.0,16
664,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,20.0,17
606,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,16.0,19
634,2266392562,98,1407133621,[1439965405],274189731,1413776458,3172394293,0,113.360001,0.0,16.0,21


In [101]:
dcs = []
for k in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{k + 1}/context.parquet')
  d['day'] = k + 1
  dcs += [d]
d = gezi.read_parquet('../input/eval/context.parquet')
d['day'] = 31
dcs += [d]
dc = pd.concat(dcs)

100%|██████████| 30/30 [00:35<00:00,  1.17s/it]


In [104]:
dcs[4][dcs[4].vid==2266392562]

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day


In [103]:
dcs[4]

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day
0,0,07c54d4eecfcccbb0503f4cc6b672741,fce8cc8ce0587de13c3fecbc13537746,183e022fe835e248ec27774c2687e312,21032c3a66d86aa1d99127d4b97cb548,5bd64e004403c21759fd4c5158e2bc04,1178034636,0,2214269418,1424707504,0,5
1,0,07c54d4eecfcccbb0503f4cc6b672741,fce8cc8ce0587de13c3fecbc13537746,183e022fe835e248ec27774c2687e312,21032c3a66d86aa1d99127d4b97cb548,5bd64e004403c21759fd4c5158e2bc04,683624456,2206654179,2214269418,1424707518,1,5
2,0,07c54d4eecfcccbb0503f4cc6b672741,fce8cc8ce0587de13c3fecbc13537746,183e022fe835e248ec27774c2687e312,21032c3a66d86aa1d99127d4b97cb548,5bd64e004403c21759fd4c5158e2bc04,1722812506,0,2214269418,1424707500,2,5
3,0,07c54d4eecfcccbb0503f4cc6b672741,fce8cc8ce0587de13c3fecbc13537746,183e022fe835e248ec27774c2687e312,21032c3a66d86aa1d99127d4b97cb548,5bd64e004403c21759fd4c5158e2bc04,3906883287,2115682546,2214269418,1424707472,3,5
4,0,07c54d4eecfcccbb0503f4cc6b672741,fce8cc8ce0587de13c3fecbc13537746,183e022fe835e248ec27774c2687e312,21032c3a66d86aa1d99127d4b97cb548,5bd64e004403c21759fd4c5158e2bc04,4250365497,1256605048,2214269418,1424707472,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2899564,0,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,3216385467,0,2092618678,1424756088,2899564,5
2899565,1,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,2254133490,0,2092618678,1424756172,2899565,5
2899566,0,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,3039103562,0,2092618678,1424756088,2899566,5
2899567,0,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,3354895296,0,2092618678,1424756088,2899567,5


In [115]:
dcs[4].tail(100)

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day
2899469,0,282c3907fb9fa42293f03eee420adbbb,9de1665e6430fa58785027dbf9ab53e0,fd9fc9f0b92891044d964d352169e497,95f0d16099fcfd8dab74600e61279f78,623faac730444c2a424e3165aa336c2e,2593433598,4114165057,1189422992,1424781854,2899469,5
2899470,0,282c3907fb9fa42293f03eee420adbbb,9de1665e6430fa58785027dbf9ab53e0,fd9fc9f0b92891044d964d352169e497,95f0d16099fcfd8dab74600e61279f78,623faac730444c2a424e3165aa336c2e,273467815,0,1189422992,1424746181,2899470,5
2899471,0,282c3907fb9fa42293f03eee420adbbb,9de1665e6430fa58785027dbf9ab53e0,fd9fc9f0b92891044d964d352169e497,95f0d16099fcfd8dab74600e61279f78,623faac730444c2a424e3165aa336c2e,997539952,1202296985,1189422992,1424746162,2899471,5
2899472,0,282c3907fb9fa42293f03eee420adbbb,9de1665e6430fa58785027dbf9ab53e0,fd9fc9f0b92891044d964d352169e497,95f0d16099fcfd8dab74600e61279f78,623faac730444c2a424e3165aa336c2e,1438669718,3097904224,1189422992,1424746182,2899472,5
2899473,0,282c3907fb9fa42293f03eee420adbbb,9de1665e6430fa58785027dbf9ab53e0,fd9fc9f0b92891044d964d352169e497,95f0d16099fcfd8dab74600e61279f78,623faac730444c2a424e3165aa336c2e,2215339506,0,1189422992,1424742138,2899473,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2899564,0,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,3216385467,0,2092618678,1424756088,2899564,5
2899565,1,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,2254133490,0,2092618678,1424756172,2899565,5
2899566,0,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,3039103562,0,2092618678,1424756088,2899566,5
2899567,0,f08b7032cfeef709893972061e8d6523,9de1665e6430fa58785027dbf9ab53e0,fb1ec50d3ce5505057f763c49d503b5a,ba195dd85894d6957bc61edca5ac7026,ac395365e055822bed2ae6a90434522f,3354895296,0,2092618678,1424756088,2899567,5


In [119]:
di[di.vid==997539952].head()

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
22287,997539952,30,1424606111,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,2234347078,0,2340.22998,0.0000,0.0,3
21358,997539952,30,1424606111,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,2234347078,0,2340.22998,0.0000,0.0,4
21671,997539952,30,1424606111,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,2234347078,0,2340.22998,0.1250,1876696.0,5
22233,997539952,30,1424606111,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,2234347078,0,2340.22998,0.1200,4926160.0,6
22124,997539952,30,1424606111,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,2234347078,0,2340.22998,0.1175,6309892.0,7


In [120]:
di[di.vid==1202296985].head()

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
5111,1202296985,90,1424087709,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,472867291,0,63.220001,0.0675,3212132.0,1
4975,1202296985,90,1424087709,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,472867291,0,63.220001,0.0575,3861484.0,2
4769,1202296985,90,1424087709,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,472867291,0,63.220001,0.0500,4394960.0,3
4576,1202296985,90,1424087709,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,472867291,0,63.220001,0.0550,4903156.0,4
4686,1202296985,90,1424087709,"[1968054038, 1268406585, 3527018506, 57973250,...",2880934939,1820266297,472867291,0,63.220001,0.0475,4069376.0,5


In [105]:
dcs[4][dcs[4].prev==2266392562]

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day
2787092,0,8d2efe1ce7b6015a159702a906b4800c,9de1665e6430fa58785027dbf9ab53e0,183e022fe835e248ec27774c2687e312,e49dc1e8e576ff35a07e1182f4e98c91,0aae1169c8d33d540e41984e1e9703de,2178715915,2266392562,613774136,1424783285,2787092,5


In [108]:
di[di.vid==683624456].head()

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
37174,683624456,50,1361505261,"[3918807355, 2105234759, 1704846244, 179524530...",3302403079,1413776458,2234347078,0,3914.189941,0.0725,20580.0,1
36159,683624456,50,1361505261,"[3918807355, 2105234759, 1704846244, 179524530...",3302403079,1413776458,2234347078,0,3914.189941,0.0825,20476.0,2
34711,683624456,50,1361505261,"[3918807355, 2105234759, 1704846244, 179524530...",3302403079,1413776458,2234347078,0,3914.189941,0.0925,19456.0,3
33119,683624456,50,1361505261,"[3918807355, 2105234759, 1704846244, 179524530...",3302403079,1413776458,2234347078,0,3914.189941,0.0925,20740.0,4
33669,683624456,50,1361505261,"[3918807355, 2105234759, 1704846244, 179524530...",3302403079,1413776458,2234347078,0,3914.189941,0.0900,22108.0,5


In [109]:
di[di.vid==2206654179].head()

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
20327,2206654179,101,1361410740,"[697164268, 2105234759, 3647308832, 1092821632...",3302403079,1413776458,472867291,0,77.389999,0.0,1632.0,1
19734,2206654179,101,1361410740,"[697164268, 2105234759, 3647308832, 1092821632...",3302403079,1413776458,472867291,0,77.389999,0.0,2072.0,2
18947,2206654179,101,1361410740,"[697164268, 2105234759, 3647308832, 1092821632...",3302403079,1413776458,472867291,0,77.389999,0.0,2672.0,3
18139,2206654179,101,1361410740,"[697164268, 2105234759, 3647308832, 1092821632...",3302403079,1413776458,472867291,0,77.389999,0.0,3208.0,4
18448,2206654179,101,1361410740,"[697164268, 2105234759, 3647308832, 1092821632...",3302403079,1413776458,472867291,0,77.389999,0.0,3536.0,5


In [112]:
di[di.vid==4250365497].head()

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
57447,4250365497,110,1397360741,"[2875188848, 1443431957, 2834403402, 393660770...",527267373,1413776458,2234347078,0,2847.080078,0.1050,65656.0,1
56083,4250365497,110,1397360741,"[2875188848, 1443431957, 2834403402, 393660770...",527267373,1413776458,2234347078,0,2847.080078,0.1075,67056.0,2
53938,4250365497,110,1397360741,"[2875188848, 1443431957, 2834403402, 393660770...",527267373,1413776458,2234347078,0,2847.080078,0.1100,72664.0,3
51331,4250365497,110,1397360741,"[2875188848, 1443431957, 2834403402, 393660770...",527267373,1413776458,2234347078,0,2847.080078,0.1150,76476.0,4
52182,4250365497,110,1397360741,"[2875188848, 1443431957, 2834403402, 393660770...",527267373,1413776458,2234347078,0,2847.080078,0.1150,85788.0,5


In [113]:
di[di.vid==1256605048].head()

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,day
39286,1256605048,87,1397749133,"[2875188848, 2131366248, 3936607703, 144343195...",527267373,1413776458,472867291,0,77.389999,0.0625,54732.0,1
38261,1256605048,87,1397749133,"[2875188848, 2131366248, 3936607703, 144343195...",527267373,1413776458,472867291,0,77.389999,0.0275,55688.0,2
36722,1256605048,87,1397749133,"[2875188848, 2131366248, 3936607703, 144343195...",527267373,1413776458,472867291,0,77.389999,0.0225,63432.0,3
35039,1256605048,87,1397749133,"[2875188848, 2131366248, 3936607703, 144343195...",527267373,1413776458,472867291,0,77.389999,0.0150,64156.0,4
35598,1256605048,87,1397749133,"[2875188848, 2131366248, 3936607703, 144343195...",527267373,1413776458,472867291,0,77.389999,0.0175,65724.0,5


In [110]:
dc[dc.prev==2266392562]

,aver,day,did,index,label,mf,mod,prev,region,sver,timestamp,vid
2787092,183e022fe835e248ec27774c2687e312,5,0aae1169c8d33d540e41984e1e9703de,2787092,0.0,9de1665e6430fa58785027dbf9ab53e0,8d2efe1ce7b6015a159702a906b4800c,2266392562,613774136,e49dc1e8e576ff35a07e1182f4e98c91,1424783285,2178715915
2683339,e452a790a877bea65da21f6dbe9d9265,6,fe8c673aa7be5d6dce8b1c294bcb83ea,2683339,0.0,932a5cc56763396c6b205a63868d6316,670f6fc0013eb09e298bd84cf68e552d,2266392562,2092618678,e49dc1e8e576ff35a07e1182f4e98c91,1424830736,2178715915
1219527,183e022fe835e248ec27774c2687e312,9,7f778ecc8932b09963f5bae5c801e38c,1219527,0.0,932a5cc56763396c6b205a63868d6316,3cb768c0a171d3c6f7279c71b0cb12df,2266392562,1196749989,77174f814122a113d1d4321b25c6f899,1425137888,2178715915
1499494,8a30ae7b3ec823c36539b1dc59882fd8,9,8a0b87dcbfe8aa6ee7beaf6f7bde28e6,1499494,0.0,9de1665e6430fa58785027dbf9ab53e0,953a23da729356b911210b6f482921cd,2266392562,2247014434,e49dc1e8e576ff35a07e1182f4e98c91,1425055651,2178715915
1702524,8a30ae7b3ec823c36539b1dc59882fd8,10,53284ada02ba2725b4759725eabc9481,1702524,0.0,9de1665e6430fa58785027dbf9ab53e0,e277978dc2149421abd2b4b7b69774f7,2266392562,959233027,ba195dd85894d6957bc61edca5ac7026,1425225034,2178715915
1683535,856984b4b943d983e1ecd063561ede2d,14,171e40b51950ff9fd2292d770d9c0b35,1683535,0.0,9de1665e6430fa58785027dbf9ab53e0,d6e65ce07d50bac6c6292115244fd024,2266392562,2504753436,ba195dd85894d6957bc61edca5ac7026,1425568449,2178715915
1831436,8a30ae7b3ec823c36539b1dc59882fd8,15,5ee4633121e24a702eb73c21242a2c03,1831436,0.0,932a5cc56763396c6b205a63868d6316,3cb768c0a171d3c6f7279c71b0cb12df,2266392562,1873721752,77174f814122a113d1d4321b25c6f899,1425651190,2178715915
3162137,09917cf8afed8861a4a87628ebb4d53f,16,c3b19bf41692d65dc4150d6cc24b79cc,3162137,0.0,9de1665e6430fa58785027dbf9ab53e0,5199d3ec3bda4877aabdb6e5cdb3db82,2266392562,807073250,e49dc1e8e576ff35a07e1182f4e98c91,1425664525,2178715915
259124,856984b4b943d983e1ecd063561ede2d,17,e89d919d4b9193b70f48a3c813520b24,259124,0.0,932a5cc56763396c6b205a63868d6316,7ba4d2fc10d03a33549e0c741db9829c,2266392562,795955078,fb7400b0ab4726a59464a94b6842f136,1425829219,2178715915
2314174,856984b4b943d983e1ecd063561ede2d,19,fb45a14c031f6ba63e2c2685beb4007f,2314174,0.0,9de1665e6430fa58785027dbf9ab53e0,a263ce13c5f2068a732f82e9b3ba264d,2266392562,3490651091,ba195dd85894d6957bc61edca5ac7026,1425955793,2178715915


In [111]:
# raw

In [123]:
dr = gezi.read_parquet('../input/train/raw.parquet')

In [124]:
dr

,vid,image_emb,title,story
0,5893106,"-8.26598072052,1.63619112968,17.9674415588,6.9...","199575,199576,40564",
1,10423101,"-3.63512468338,2.38069868088,14.9096345901,1.2...","717361,318",
2,15819492,"0.107854895294,3.20008993149,20.3623485565,-0....","396,1705,31495,3424,77035,85420,28,17970,9300,...",
3,16814925,"-11.0617599487,1.99434304237,14.9591712952,1.2...","8579,18147,73,227",
4,20360674,"6.6802148819,1.83605599403,15.9008598328,3.244...","22485,118989,12418,396611,61891,3029,26353,396...",
...,...,...,...,...
183788,4276491220,"-4.29484510422,6.66334295273,16.4290809631,-4....","3072,34976,173488,1864,6276,492635,58,19175,24...",
183789,4283547666,"-4.81997919083,13.6302375793,14.3650541306,6.9...","266947,7450,58397,42,58167,44,2577,5",
183790,4283640696,"-0.665687978268,11.5875854492,9.86795520782,3....","406009,91149,52611,66378,290,355096",
183791,4290887658,"2.51159644127,2.58366537094,9.91071224213,4.36...","42,128095,44,188981,28,6705,87381,2679,59,3316...",


In [125]:
dr[dr.vid==2266392562]

,vid,image_emb,title,story
2257,2266392562,"3.34597158432,8.28211307526,13.9855890274,-2.3...","524,859,26198,7844,443324,42,491609,44,3233,12...",
